In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("banking project").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/18 12:44:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import *

# Loading Datasets

In [6]:
loanDF = spark.read.csv("loan.csv", inferSchema =True, header=True)

loanDF.printSchema()
loanDF.show()

root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Family Size: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Expenditure: integer (nullable = true)
 |-- Use Frequency: integer (nullable = true)
 |-- Loan Category: string (nullable = true)
 |-- Loan Amount: string (nullable = true)
 |-- Overdue: integer (nullable = true)
 |--  Debt Record: string (nullable = true)
 |--  Returned Cheque: integer (nullable = true)
 |--  Dishonour of Bill: integer (nullable = true)

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Re

In [7]:
print(loanDF.count() ," distinct " , loanDF.distinct().count() )

500  distinct  500


In [9]:
loanDF.groupBy("Loan Category").count().orderBy(col("count").desc()).show(truncate=False)

+------------------+-----+
|Loan Category     |count|
+------------------+-----+
|GOLD LOAN         |77   |
|HOUSING           |67   |
|AUTOMOBILE        |60   |
|TRAVELLING        |53   |
|RESTAURANTS       |41   |
|COMPUTER SOFTWARES|35   |
|SHOPPING          |35   |
|BUSINESS          |24   |
|EDUCATIONAL LOAN  |20   |
|RESTAURANT        |20   |
|HOME APPLIANCES   |14   |
|DINNING           |14   |
|ELECTRONICS       |14   |
|AGRICULTURE       |12   |
|BOOK STORES       |7    |
|BUILDING          |7    |
+------------------+-----+



# Trying out some commands 

In [11]:
loanDF.filter(col("Expenditure")>50000).show(truncate=False)
loanDF.filter(col("Expenditure")>50000).count()



+-----------+---+------+---------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|Occupation     |Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+---------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|IB14158    |54 |MALE  |AIRPORT OFFICER|MARRIED       |6          |80000 |62541      |2            |AUTOMOBILE   | 20,45,789 |1      |16,599      |2               |3                 |
|IB14176    |54 |MALE  |AIRPORT OFFICER|MARRIED       |6          |80000 |62541      |2            |HOUSING      | 20,45,789 |1      |16,599      |2               |3                 |
|IB14204    |54 |MALE  |AIRPORT OFFICER|MARRIED       |6          |81000 |62541 

6

In [15]:
loanDF.filter(col("Family Size")>3).show(truncate=False)

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|Occupation         |Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category     |Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|IB14001    |30 |MALE  |BANK MANAGER       |SINGLE        |4          |50000 |22199      |6            |HOUSING           | 10,00,000 |5      |42,898      |6               |9                 |
|IB14008    |44 |MALE  |PROFESSOR          |MARRIED       |6          |51000 |19999      |4            |SHOPPING          |50,000     |3      |33,999      |1               |5                 |
|IB14018    |29 |MALE  |TEACHER    

In [16]:
txnDF = spark.read.csv("txn.csv", inferSchema=True, header =True)


In [17]:
txnDF.printSchema()

root
 |-- Account No: string (nullable = true)
 |-- TRANSACTION DETAILS: string (nullable = true)
 |-- VALUE DATE: string (nullable = true)
 |--  WITHDRAWAL AMT : double (nullable = true)
 |--  DEPOSIT AMT : double (nullable = true)
 |-- BALANCE AMT: double (nullable = true)



In [20]:
txnDF.groupBy(col("Account No")).count().show(truncate=False)

+-------------+-----+
|Account No   |count|
+-------------+-----+
|409000438611'|4588 |
|1196711'     |10536|
|1196428'     |48779|
|409000493210'|6014 |
|409000611074'|1093 |
|409000425051'|802  |
|409000405747'|51   |
|409000493201'|1044 |
|409000438620'|13454|
|409000362497'|29840|
+-------------+-----+



In [21]:
txnDF.groupBy("Account No").max(" WITHDRAWAL AMT ").orderBy("max( WITHDRAWAL AMT )", ascending = False).show()



+-------------+---------------------+
|   Account No|max( WITHDRAWAL AMT )|
+-------------+---------------------+
|     1196711'|        4.594475464E8|
|409000438620'|                4.0E8|
|409000425051'|               3.54E8|
|409000438611'|                2.4E8|
|409000405747'|                1.7E8|
|     1196428'|                1.5E8|
|409000362497'|        1.413662392E8|
|409000493210'|                1.5E7|
|409000493201'|            2500000.0|
|409000611074'|             912000.0|
+-------------+---------------------+



In [22]:
txnDF.groupBy("Account No").min(" WITHDRAWAL AMT ").orderBy("min( WITHDRAWAL AMT )").show()


+-------------+---------------------+
|   Account No|min( WITHDRAWAL AMT )|
+-------------+---------------------+
|409000493210'|                 0.01|
|409000438611'|                  0.2|
|     1196711'|                 0.25|
|     1196428'|                 0.25|
|409000438620'|                 0.34|
|409000362497'|                 0.97|
|409000425051'|                 1.25|
|409000493201'|                  2.1|
|409000405747'|                 21.0|
|409000611074'|                120.0|
+-------------+---------------------+



In [23]:

txnDF.groupBy("Account No").max(" DEPOSIT AMT ").orderBy("max( DEPOSIT AMT )", ascending = False).show()


+-------------+------------------+
|   Account No|max( DEPOSIT AMT )|
+-------------+------------------+
|409000438620'|           5.448E8|
|     1196711'|             5.0E8|
|     1196428'|     2.119594422E8|
|409000405747'|           2.021E8|
|409000362497'|             2.0E8|
|409000438611'|          1.7025E8|
|409000425051'|             1.5E7|
|409000493210'|             1.5E7|
|409000611074'|         3000000.0|
|409000493201'|         1000000.0|
+-------------+------------------+



In [24]:
txnDF.groupBy("VALUE DATE").count().orderBy("count", ascending = False).show()

+----------+-----+
|VALUE DATE|count|
+----------+-----+
| 27-Jul-17|  567|
| 13-Aug-18|  463|
|  8-Nov-17|  402|
|  7-Oct-17|  382|
| 10-Jul-18|  374|
| 12-Dec-17|  367|
| 12-Sep-18|  365|
|  9-Aug-18|  360|
| 19-Sep-17|  358|
| 16-Mar-17|  353|
| 10-Sep-18|  344|
| 14-Jul-17|  333|
|  7-Mar-18|  319|
| 11-Oct-18|  303|
| 22-Aug-17|  301|
|  9-Jan-18|  299|
|  9-Oct-18|  297|
| 20-Apr-18|  296|
|  9-Jul-18|  292|
|  7-Apr-18|  291|
+----------+-----+
only showing top 20 rows



In [27]:
txnDF.select("Account No","TRANSACTION DETAILS"," WITHDRAWAL AMT ").filter(txnDF[" WITHDRAWAL AMT "]>100000).show(truncate=False)


+-------------+--------------------------------+----------------+
|Account No   |TRANSACTION DETAILS             | WITHDRAWAL AMT |
+-------------+--------------------------------+----------------+
|409000611074'|INDO GIBL Indiaforensic STL01071|133900.0        |
|409000611074'|INDO GIBL Indiaforensic STL04071|195800.0        |
|409000611074'|INDO GIBL Indiaforensic STL10071|143800.0        |
|409000611074'|INDO GIBL Indiaforensic STL11071|331650.0        |
|409000611074'|INDO GIBL Indiaforensic STL12071|129000.0        |
|409000611074'|INDO GIBL Indiaforensic STL13071|230013.0        |
|409000611074'|INDO GIBL Indiaforensic STL14071|367900.0        |
|409000611074'|INDO GIBL Indiaforensic STL15071|108000.0        |
|409000611074'|INDO GIBL Indiaforensic STL17071|141000.0        |
|409000611074'|INDO GIBL Indiaforensic STL22071|206000.0        |
|409000611074'|INDO GIBL Indiaforensic STL02081|242300.0        |
|409000611074'|INDO GIBL Indiaforensic STL04081|113250.0        |
|409000611